## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-16-51-33 +0000,bbc,Sarah Ferguson dropped from multiple charities...,https://www.bbc.com/news/articles/cddmr6v0jpzo...
1,2025-09-22-16-49-26 +0000,bbc,"Lib Dems have moral duty to defeat Farage, say...",https://www.bbc.com/news/articles/c33rp46nr1jo...
2,2025-09-22-16-49-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
3,2025-09-22-16-47-09 +0000,nyt,"As U.N. General Assembly Gathers, Here’s What ...",https://www.nytimes.com/2025/09/22/world/ameri...
4,2025-09-22-16-47-01 +0000,nyt,American Investors Will License and Oversee Ti...,https://www.nytimes.com/2025/09/22/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,48
97,kirk,34
96,charlie,27
99,memorial,17
596,he,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
204,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,154
103,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,151
196,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,147
166,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,146
211,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
204,154,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...
82,39,2025-09-22-13-00-00 +0000,wsj,Atlanta Fed President Raphael Bostic said infl...,https://www.wsj.com/economy/central-banking/fe...
205,38,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
33,37,2025-09-22-15-36-02 +0000,cbc,"Ford government lays out demands for bail, sen...",https://www.cbc.ca/news/canada/toronto/ontario...
166,37,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...
35,35,2025-09-22-15-31-23 +0000,nypost,Howard Stern blasts ABC suspension of Jimmy Ki...,https://nypost.com/2025/09/22/media/howard-ste...
57,29,2025-09-22-14-38-55 +0000,nypost,Trump to sign TikTok deal giving US majority o...,https://nypost.com/2025/09/22/us-news/trump-to...
48,26,2025-09-22-14-53-29 +0000,nypost,Sarah Ferguson dumped by children’s hospice ch...,https://nypost.com/2025/09/22/world-news/sarah...
211,26,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...
159,26,2025-09-22-06-10-20 +0000,nypost,Two 25-year-old elk hunters found dead with no...,https://nypost.com/2025/09/22/us-news/two-elk-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
